<a href="https://colab.research.google.com/github/mcgmed/Brain-Tumor-Classification-from-MRI-Images/blob/main/brain_tumor_classification_from_mri_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 pillow \
                                 numpy

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.2 which is incompatible.


{'status': 'ok', 'restart': True}

In [1]:
PROJECT_ID = "project-id"
REGION = "us-central1"

! gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
BUCKET_URI = f"gs://bucket-{PROJECT_ID}"
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

In [4]:
import os
import numpy as np
import tensorflow as tf
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [5]:
TRAIN_VERSION = "tf-cpu.2-9"
DEPLOY_VERSION = "tf2-cpu.2-9"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

In [6]:
JOB_NAME = "brain_tumor_classification_from_mri_images"
MODEL_DIR = "{}/{}".format(BUCKET_URI, JOB_NAME)

TRAIN_STRATEGY = "single"
EPOCHS = 200

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--distribute=" + TRAIN_STRATEGY,
]

In [7]:
%%writefile task.py

import os
import sys
import pathlib
import argparse
import tensorflow as tf
from tensorflow.python.client import device_lib

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=200, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=4570//200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BATCH_SIZE = 32
IMG_HEIGHT = 180
IMG_WIDTH = 180

TRAINING_DATA_DIR = 'gs://bucket-atomic-griffin-394806/brain-tumor-mri-dataset/Training/'

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

train_ds = tf.keras.utils.image_dataset_from_directory(
  TRAINING_DATA_DIR,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=GLOBAL_BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  TRAINING_DATA_DIR,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=GLOBAL_BATCH_SIZE)

# Class Weights Calculation

class_names = train_ds.class_names
class_name_to_label = {class_name: idx for idx, class_name in enumerate(class_names)}
class_counts = {}

for _, labels in train_ds:
    for label in labels.numpy():
        class_name = class_names[label]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1

label_list = []
for _, labels in train_ds:
    label_list.extend(labels.numpy())

total_samples = len(label_list)
class_weights = {class_name_to_label[class_name]: total_samples / count for class_name, count in class_counts.items()}
sum_class_weights = sum(class_weights.values())
class_weights = {label: weight / sum_class_weights for label, weight in class_weights.items()}

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Build the Keras model
def build_and_compile_cnn_model():
  data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1)
  ])
  model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    data_augmentation,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(4, activation='softmax')
  ])

  def precision(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + tf.keras.backend.epsilon())

  def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + tf.keras.backend.epsilon())

  model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy(), precision, recall])

  return model

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True)
lr_sch = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.1, verbose=1, min_lr=5e-10)

# Train the model
MODEL_DIR = os.getenv("AIP_MODEL_DIR")

with strategy.scope():
    model = build_and_compile_cnn_model()

model.fit(train_ds, validation_data=val_ds,
          epochs=args.epochs,
          class_weight=class_weights,
          callbacks=[early_stopping, lr_sch])

model.save(MODEL_DIR)

Writing task.py


In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name=JOB_NAME,
    script_path="task.py",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

MODEL_DISPLAY_NAME = "brain_mri_tumor_classification"

model = job.run(
    model_display_name=MODEL_DISPLAY_NAME,
    args=CMDARGS,
    replica_count=1,
)

In [ ]:
DEPLOYED_NAME = "brain_mri_classification_deployed"
TRAFFIC_SPLIT = {"0": 100}
MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

In [ ]:
! gsutil -m cp -r gs://bucket-project-id/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0000.jpg .
! gsutil -m cp -r gs://bucket-project-id/brain-tumor-mri-dataset/Training/meningioma/Tr-meTr_0000.jpg .
! gsutil -m cp -r gs://bucket-project-id/brain-tumor-mri-dataset/Testing/notumor/Te-noTr_0000.jpg .
! gsutil -m cp -r gs://bucket-project-id/brain-tumor-mri-dataset/Testing/pituitary/Te-piTr_0000.jpg .

In [29]:
IMG_HEIGHT = 180
IMG_WIDTH = 180
BATCH_SIZE=32
example='Tr-meTr_0000.jpg'

def predict_image(example):
  img = tf.keras.utils.load_img(example, target_size=(IMG_HEIGHT, IMG_WIDTH))
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  sample = img_array.numpy().tolist() # Vertex AI endpoints do not accept ndarray and array types as input. Rather, you need to convert it into a python list.

  class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
  prediction = endpoint.predict(sample)
  score = tf.nn.softmax(prediction[0])
  print('This image most likely belongs to "{}" with a {:.2f} percent confidence.'.format(class_names[np.argmax(score)], 100 * np.max(score)))

In [30]:
example = 'Te-glTr_0000.jpg'
predict_image(example)

This image most likely belongs to "glioma" with a 34.17 percent confidence.


In [31]:
example = 'Tr-meTr_0000.jpg'
predict_image(example)

This image most likely belongs to "meningioma" with a 32.82 percent confidence.


In [32]:
example = 'Te-noTr_0000.jpg'
predict_image(example)

This image most likely belongs to "notumor" with a 35.79 percent confidence.


In [33]:
example = 'Te-piTr_0000.jpg'
predict_image(example)

This image most likely belongs to "pituitary" with a 47.46 percent confidence.


In [ ]:
endpoint.undeploy_all()

delete_bucket = False

# Delete the training job
job.delete()

endpoint.delete()
model.delete()

#if delete_bucket or os.getenv("IS_TESTING"):
#    ! gsutil rm -r $BUCKET_URI

In [34]:
print('continue')

continue
